#构建generaor
def my_generator(data,index,batch_size):
    
    data_x = data[0]
    data_y = data[1]
    #print(data_x)
    #print(data_y)
    imgs = np.zeros((batch_size,224,224,3))
    labels = np.zeros((batch_size,))
    means=[103.939, 116.779, 123.68]#imagenet 预训练模型的均值
    cv_imgs = []
    
    #index：训练集总数
#    print(((batch_size*index)-(batch_size*(index-1))))
#    print(data_x)
    for i in range(index):
        #print(type(data_x[1]))
        #print(i)
        img = cv2.imread(str(data_x[i]))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(224,224))
        cv_imgs.append(img)
        imgs[i] = img
        labels[i] = data_y[i]
    #print(len(cv_imgs))
    
    imgs -= means
#    y = labels
    y = keras.utils.to_categorical(labels, 10)
    return imgs,y

In [2]:
#加载测试集图片
def load_test():
    import os
    
    test_imgs_name = []
    for img_name in os.listdir(r'test'):
        test_imgs_name.append('test'+'//'+img_name)
        
    return test_imgs_name

input_tensor = Input(shape=(224, 224, 3))
# precess_input = Lambda(res_preprocess_input)(input_tensor)

# x = resnet_50.output
# x = Dropout(0.8)(x)
# predictions = Dense(10,activation='softmax')(x)

#  224 x 224
resModel = keras.applications.resnet50.ResNet50(include_top=True,input_tensor=input_tensor,weights='imagenet',input_shape=(224,224,3))
# 只训练最后一层
resModel = Model(inputs=resModel.input,outputs=resModel.layers[-2].output)
# origin output
origin_output = resModel.output
# origin_output = Dropout(0.5)(origin_output)
	# print(resModel.layers)
	# full connected
	# flatten = GlobalAveragePooling2D()(origin_output)
	# flatten = Dropout(0.5)(flatten)
	# 可以考虑加一个 dropout 层
	# fc_cus_layer = Dense(1000,activation='relu')(flatten)
	# print(fc_cus_layer)
	# custom output layer
	# kernel_regularizer=regularizers.l2(0.01) ,kernel_regularizer=regularizers.l2(0.1),kernel_regularizer=regularizers.l2(0.05)
prediction_layer = Dense(10, activation='softmax',name='res_out_put')(origin_output)#resModel.layers[-1].output) # origin_output
	# print(prediction_layer)
	# fine-tune model
model = Model(inputs=resModel.input, outputs=prediction_layer)

for layer in resModel.layers:
	layer.trainable = False
	# compile
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

	# fit
history1 = model.fit_generator(train_generator,len(train_imgs) // batch_size,
                               epochs=epochs,verbose=1,
                               validation_data=valid_generator,
                               validation_steps=len(valid_imgs) // batch_size)
	# 第二次训练
	# 开放接下来所有层  
for layer in model.layers[:]:
	layer.trainable = True

sgd = keras.optimizers.SGD(lr=0.001, momentum=0.9)
	# Adam
#adam = Adam(lr=0.001)
	#Adadelta()
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
	# 提前停止
#early = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=5, verbose=0, mode='auto')
history2 = model.fit_generator(train_generator,len(train_imgs) // batch_size,
                    epochs=epochs,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_imgs) // batch_size)
	#predict,需要转换为 test
# score = model.evaluate(valid_image, valid_label,batch_size = batch_size, verbose=1)

In [1]:
import numpy as np
import os
import csv
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import keras
from sklearn.cross_validation import train_test_split
from generator import DXGenerator
from testGenerator import TestGenerator

#获取数据
def get_data():
    reader = pd.read_csv('driver_imgs_list.csv')

    driver_ids =np.array(reader['subject'])
    driver_classes = reader['classname']
    driver_imgs = reader['img']

    driver_id_dict={}
    
    for index in range(len(driver_ids)):
        if driver_ids[index] in driver_id_dict.keys():
            driver_id_dict[driver_ids[index]].append((driver_classes[index], driver_imgs[index]))
        else:
            driver_id_dict[driver_ids[index]] = [(driver_classes[index], driver_imgs[index])]

    
    #train
#     train_img_name=[]
#     train_labels_count=0
#     id_key_list = list(driver_id_dict.keys())
#     np.random.shuffle(id_key_list)
#     for index in id_key_list[:-2]:
#         train_labels_count += len(driver_id_dict[index])
    
#     train_dict = {}
#     train_image_labels = np.zeros((train_labels_count,),dtype=np.int)
# #    print(len(train_image_labels))
#     for index in id_key_list[:-2]:
#         if index in driver_id_dict.keys():
#             for i in range(len(driver_id_dict[index])):
#                 img_name = 'train' + '//' + str((driver_id_dict[index][i])[0]) + '//' + str((driver_id_dict[index][i])[1])
#                 train_dict[img_name] = int((driver_id_dict[index][i])[0][1:])
    
#     for i in range(len(list(train_dict.items()))):
#         train_img_name.append(list(train_dict.items())[i][0])
#         train_image_labels[i] = list(train_dict.items())[i][1]
# #    print(len(train_image_labels))
    
#     train = list(zip(train_img_name,train_image_labels))
#     train_img_name=[]
#     np.random.shuffle(train)
#     for i in range(len(train)):
#         train_img_name.append(train[i][0])
#         train_image_labels[i] = train[i][1]
# #    print(len(train_image_labels))
        
    
#     #test
#     valid_img_name=[]
#     valid_labels_count=0
#     for index in id_key_list[-2:]:
#         valid_labels_count += len(driver_id_dict[index])
    
#     valid_image_labels = np.zeros((valid_labels_count,),dtype=np.int)
#     valid_dict={}
#     for index in id_key_list[-2:]:

#         if index in driver_id_dict.keys():
#             for i in range(len(driver_id_dict[index])):
#                 img_name = 'train' + '//' + str((driver_id_dict[index][i])[0]) + '//' + str((driver_id_dict[index][i])[1])
#                 valid_dict[img_name]=int((driver_id_dict[index][i])[0][1:])

#     for i in  range(len(list(valid_dict.items()))):
#         valid_img_name.append(list(valid_dict.items())[i][0])
#         valid_image_labels[i]=list(valid_dict.items())[i][1]
        
#     print(list(valid_dict.items())[i][0],valid_image_labels[i],list(valid_dict.items())[i][1])
#     print(valid_image_labels)        

#根据ID分割司机的数据集
    key_2_number = {'c0':0,'c1':1,'c2':2,'c3':3,'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
    valid_img_name = []
    valid_image_labels = []
    train_img_name = []
    train_image_labels = []
    ids = driver_id_dict.keys()
    driver_count = len(ids) #总共多少司机
    # 取 /2 /3 的位置的司机
    validation_id = [int(driver_count / 2),int(driver_count / 3)]
    index = 0 
    # 寻找图片,保存图片名
    for driver_id in ids:
        # index 代表去第几个位置的司机
        for driver_class,driver_imageName in driver_id_dict[driver_id]:
            if index in validation_id:
                # valid
                valid_img_name.append('train/' + str(driver_class) + '/' + str(driver_imageName))
                valid_image_labels.append(key_2_number[str(driver_class)])
            else:
                # train
                train_img_name.append('train/' + str(driver_class) + '/' + str(driver_imageName))
                train_image_labels.append(key_2_number[str(driver_class)])
        index += 1


    return train_img_name,valid_img_name,train_image_labels,valid_image_labels
#total_classes = len(os.listdir('train'))
# get_data()


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the 

In [2]:
batch_size = 32
epochs = 8
classes = [0,1,2,3,4,5,6,7,8,9]
train_imgs,valid_imgs,train_labels,valid_labels = get_data()
#print(valid_imgs[:500],valid_labels[:500])
# print(len(train_imgs),len(train_labels),len(valid_imgs),len(valid_labels))
#train_x,train_y = my_generator((train_imgs,train_labels),len(train_imgs),batch_size)
#valid_x,valid_y = my_generator((valid_imgs,valid_labels),len(valid_imgs),batch_size)


In [3]:
#构建网络
from keras.applications import vgg19,resnet50,inception_v3,xception
from keras.models import Sequential,Model
from keras.layers import Input,Convolution2D,MaxPooling2D,Dense,Activation,Dropout,Flatten,GlobalAveragePooling2D
import h5py
from keras.applications.resnet50 import preprocess_input as res_preprocess_input
from keras.applications.xception import preprocess_input as xcep_preprocess_input
from keras.applications.inception_v3 import preprocess_input as incep_preprocess_input
from keras import regularizers
from keras.callbacks import ModelCheckpoint

In [6]:

#vgg_19 = vgg19.VGG19(include_top=False, weights='imagenet',input_shape=(224,224,3),pooling='avg')
#vgg_19.summary()

#model_img = keras.utils.plot_model(vgg_19,to_file='model.png',show_shapes=False,show_layer_names=True,rankdir='TB')

#resnet_50.summary()

#inc_v3 = inception_v3.InceptionV3(include_top=False, weights='imagenet',input_shape=(224,224,3),pooling='avg')
#inc_v3.summary()

# x_ception = xception.Xception(include_top=False, weights='imagenet',input_shape=(224,224,3),pooling='avg')
# x_ception.summary()

train_generator = DXGenerator((train_imgs,train_labels),len(train_imgs),
                               des_size=(299,299),
                               is_directory=True,batch_size=batch_size,
                               shuffle=True,seed=0)
train_generator.name="train"

valid_generator = DXGenerator((valid_imgs,valid_labels),len(valid_imgs),
                               des_size=(299,299),
                               is_directory=True,batch_size=batch_size,
                               shuffle=True,seed=0)
valid_generator.name="valid"

Input = keras.layers.Input(shape=(299,299,3))
process_input = keras.layers.Lambda(xcep_preprocess_input)(Input)

# resnet_50 = resnet50.ResNet50(include_top=False, weights='imagenet',input_tensor=process_input,pooling='avg')
x_ception = xception.Xception(include_top=False, weights='imagenet',input_tensor=process_input,pooling='avg')
resModel = Model(inputs=x_ception.input,outputs=x_ception.output)
#函数式模型

output = resModel.output

#为了防止过拟合，加入dropout层和正则化
output = Dropout(0.8)(output)
#,kernel_regularizer=regularizers.l2(0.01)
predictions = Dense(10,activation='softmax',name='driver_classfier')(output)
# print(predictions)
model = Model(inputs=resModel.input,outputs=predictions)

#先使用resnet最后一层进行训练，防止权重波动太大
for layers in resModel.layers[:-5]:
    layers.trainable=False

sgd = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
adam = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit_generator(train_generator,len(train_imgs) // batch_size,
                    epochs=2,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_imgs)// batch_size)

#然后放开resnet所有层进行训练
for layers in model.layers:
    layers.trainable=True
    
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit_generator(train_generator,len(train_imgs) // batch_size,
                    epochs=10,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_imgs) // batch_size)

#print(train_x,'\n',train_y)model.evaluate(x_test, rain 0 0y_test, batch_size=32)
#score = model.evaluate(v_x, v_y, batch_size=32)
#predictions = model.predict(v_x)
#print(score)


Epoch 1/2
640/640 [==============================] - 491s 768ms/step - loss: 0.4418 - acc: 0.8805 - val_loss: 2.5584 - val_acc: 0.3641
Epoch 2/2
640/640 [==============================] - 486s 759ms/step - loss: 0.0928 - acc: 0.9764 - val_loss: 4.1157 - val_acc: 0.3161
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[32,1024,19,19] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: block13_sepconv2_bn_1/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, data_format="NCHW", epsilon=0.001, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block13_sepconv2_1/separable_conv2d, block13_sepconv2_bn_1/gamma/read, block13_sepconv2_bn_1/beta/read, block13_sepconv1_bn_1/Const_4, block13_sepconv1_bn_1/Const_4)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_2/mul/_5341 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_15747_loss_2/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint

In [5]:
#Resnet50
train_generator = DXGenerator((train_imgs,train_labels),len(train_imgs),
                               is_directory=True,batch_size=batch_size,
                               shuffle=True,seed=0)
train_generator.name="train"

valid_generator = DXGenerator((valid_imgs,valid_labels),len(valid_imgs),
                               is_directory=True,batch_size=batch_size,
                               shuffle=True,seed=0)
valid_generator.name="valid"


Input = keras.layers.Input(shape=(224,224,3))
process_input = keras.layers.Lambda(res_preprocess_input)(Input)
resnet_50 = resnet50.ResNet50(include_top=True, weights='imagenet',input_tensor=process_input,pooling='avg')
# resnet_50.summary()
resModel = Model(inputs=resnet_50.input,outputs=resnet_50.layers[-2].output)
#函数式模型
output = resModel.output

#为了防止过拟合，加入dropout层和正则化
output = Dropout(0.6)(output)
#,kernel_regularizer=regularizers.l2(0.01)
predictions = Dense(10,activation='softmax',name='driver_classfier')(output)
# print(predictions)
model = Model(inputs=resModel.input,outputs=predictions)

#先使用resnet最后一层进行训练，防止权重波动太大
for layers in resnet_50.layers:
    layers.trainable=False

sgd = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
adam = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit_generator(train_generator,len(train_imgs) // batch_size,
                    epochs=1,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_imgs)// batch_size)

#然后放开resnet所有层进行训练
for layers in resModel.layers:
    layers.trainable=True
    
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_check=ModelCheckpoint("resnet50.h5", monitor='val_loss', 
                            save_best_only=True, save_weights_only=False, mode='auto', period=1)

model.fit_generator(train_generator,len(train_imgs) // batch_size,
                    epochs=10,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_imgs) // batch_size,
                   callbacks=[model_check])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        lambda_2[0][0]                   
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Epoch 1/1
640/640 [==============================] - 213s 333ms/step - loss: 1.2107 - acc: 0.5988 - val_loss: 3.0544 - val_acc: 0.2516
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
model.save_weights("resnet.h5")

In [5]:
#inceptionV3


train_generator = DXGenerator((train_imgs,train_labels),len(train_imgs),
                               des_size=(299,299),
                               is_directory=True,batch_size=batch_size,
                               shuffle=True,seed=0)
train_generator.name="train"

valid_generator = DXGenerator((valid_imgs,valid_labels),len(valid_imgs),
                               des_size=(299,299),
                               is_directory=True,batch_size=batch_size,
                               shuffle=True,seed=0)
valid_generator.name="valid"




Input = keras.layers.Input(shape=(299,299,3))
process_input = keras.layers.Lambda(incep_preprocess_input)(Input)
inc_v3 = inception_v3.InceptionV3(include_top=False, weights='imagenet',input_tensor=process_input)
#函数式模型
output = inc_v3.output

#为了防止过拟合，加入dropout层和正则化
output = Dropout(0.3)(output)
#,kernel_regularizer=regularizers.l2(0.01)
pooling_layer = GlobalAveragePooling2D()(output)
fc_layer = Dense(1024, activation='relu')(pooling_layer)
predictions = Dense(10,activation='softmax',name='driver_classfier')(fc_layer)
# print(predictions)
model = Model(inputs=inc_v3.input,outputs=predictions)

#先使用resnet最后一层进行训练，防止权重波动太大
for layers in model.layers:
    layers.trainable=False

sgd = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
adam = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit_generator(train_generator,len(train_imgs) // batch_size,
                    epochs=2,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_imgs)// batch_size)

#然后放开resnet所有层进行训练
for layers in inc_v3.layers:
    layers.trainable=True
    
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model_check=ModelCheckpoint("inc_v3.h5", monitor='val_loss', 
                            save_best_only=True, save_weights_only=False, mode='auto', period=3)

model.fit_generator(train_generator,len(train_imgs) // batch_size,
                    epochs=8,verbose=1,
                    validation_data=valid_generator,
                    validation_steps=len(valid_imgs) // batch_size,
                   callbacks=[model_check])

Epoch 1/2
640/640 [==============================] - 271s 423ms/step - loss: 2.4693 - acc: 0.1120 - val_loss: 2.5714 - val_acc: 0.1109
Epoch 2/2
640/640 [==============================] - 261s 407ms/step - loss: 2.4687 - acc: 0.1120 - val_loss: 2.5714 - val_acc: 0.1109
Epoch 1/8
640/640 [==============================] - 813s 1s/step - loss: 1.1534 - acc: 0.5668 - val_loss: 1.4200 - val_acc: 0.6120
Epoch 2/8
640/640 [==============================] - 790s 1s/step - loss: 0.1665 - acc: 0.9523 - val_loss: 2.1817 - val_acc: 0.5583
Epoch 3/8
640/640 [==============================] - 841s 1s/step - loss: 0.0815 - acc: 0.9778 - val_loss: 0.2273 - val_acc: 0.9276
Epoch 4/8
640/640 [==============================] - 791s 1s/step - loss: 0.0565 - acc: 0.9847 - val_loss: 0.5146 - val_acc: 0.8542
Epoch 5/8
640/640 [==============================] - 791s 1s/step - loss: 0.0518 - acc: 0.9850 - val_loss: 0.9324 - val_acc: 0.8224
Epoch 6/8
640/640 [==============================] - 791s 1s/step - lo

In [8]:
model.save_weights('inc_v3_weights.h5')

KeyboardInterrupt: 